We can start by importing the packages we'll be using in this exploration.

In [1]:
# Import relevant packages
import os
import pandas as pd

Next, lets load in our dataset from Kaggle. It consists of 6 files, which we will import as DataFrames from their respective filetypes.

In [2]:
# Load HR data sourced from Kaggle 
# Source: https://www.kaggle.com/datasets/vjchoudhary7/hr-analytics-case-study

data_dictionary = pd.read_excel('data_dictionary.xlsx')
emp_survey = pd.read_csv('employee_survey_data.csv')
man_survey = pd.read_csv('manager_survey_data.csv')
in_time = pd.read_csv('in_time.csv')
out_time = pd.read_csv('out_time.csv')
gen_data = pd.read_csv('general_data.csv')

Let's check out the largest of these DataFrames, 'gen_data'.

In [3]:
# Print info
print(gen_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   Attrition                4410 non-null   object 
 2   BusinessTravel           4410 non-null   object 
 3   Department               4410 non-null   object 
 4   DistanceFromHome         4410 non-null   int64  
 5   Education                4410 non-null   int64  
 6   EducationField           4410 non-null   object 
 7   EmployeeCount            4410 non-null   int64  
 8   EmployeeID               4410 non-null   int64  
 9   Gender                   4410 non-null   object 
 10  JobLevel                 4410 non-null   int64  
 11  JobRole                  4410 non-null   object 
 12  MaritalStatus            4410 non-null   object 
 13  MonthlyIncome            4410 non-null   int64  
 14  NumCompaniesWorked      

Let's also take a look at the metadata file to make sure we understand what the 'gen_data' columns are referencing.

In [4]:
# Print metadata
print(data_dictionary)

                    Variable  \
0                        Age   
1                  Attrition   
2             BusinessTravel   
3                 Department   
4           DistanceFromHome   
5                  Education   
6                        NaN   
7                        NaN   
8                        NaN   
9                        NaN   
10            EducationField   
11             EmployeeCount   
12            EmployeeNumber   
13   EnvironmentSatisfaction   
14                       NaN   
15                       NaN   
16                       NaN   
17                    Gender   
18            JobInvolvement   
19                       NaN   
20                       NaN   
21                       NaN   
22                  JobLevel   
23                   JobRole   
24           JobSatisfaction   
25                       NaN   
26                       NaN   
27                       NaN   
28             MaritalStatus   
29             MonthlyIncome   
30      

Now we can begin with some housekeeping: lets save a copy of the raw general dataframe for later reference, and also set a common index between DataFrames where possible.

In [5]:
# Save a copy of the original dataframe for later reference
gen_data_orig = gen_data.copy()

# Set indexes to Employee ID
gen_data = gen_data.set_index('EmployeeID')
emp_survey = emp_survey.set_index('EmployeeID')
man_survey = man_survey.set_index('EmployeeID')

Focusing on the general HR DataFrame, lets do some initial cleaning tasks, like calculating the number of missing values for each column and removing records with missing values.

In [6]:
# Missing value counts for general data
print(gen_data.isna().sum())

# Remove records with missing values
gen_data.dropna(subset=['NumCompaniesWorked','TotalWorkingYears'], inplace=True)

# Reprint missing value counts to ensure removal of missing records
print(gen_data.isna().sum())

Age                         0
Attrition                   0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
Over18                      0
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int64
Age                        0
Attrition                  0
BusinessTravel             0
Department                 0
DistanceFromHome           0
Education                  0
EducationField             0
EmployeeCount              0
Gender                     0
JobLevel                   0
JobRole

Next, lets convert non-US units to those used in the United States.

In [7]:
# Convert commute distance to miles (mi) from kilometers (km)
gen_data['DistanceFromHome'] = gen_data['DistanceFromHome']*0.621371

# Convert monthly income to United States Dollar (USD) from Indian Rupee (INR)
gen_data['MonthlyIncome'] = gen_data['MonthlyIncome']*0.012900993 

Focusing now on the two timesheet DataFrames, lets again start with initial evaluation and cleaning tasks, like calculating the number of missing values for each column and removing holidays and weekends (dates in which all employees were not present at work).

In [8]:
# --- WORKING WITH TIMESHEET DATA --- #

# Missing value counts for timesheet data
print(in_time.isna().sum())
print(out_time.isna().sum())

# Find working days and remove holidays
in_time_workdays = in_time.T.dropna(how="all")
out_time_workdays  = out_time.T.dropna(how="all")

# Transpose the DataFrames for later manipulations
in_time_workdays  = in_time_workdays.T
out_time_workdays  = out_time_workdays.T

# Drop irrelevant columns
in_time_workdays = in_time_workdays.drop(labels=['Unnamed: 0'], axis=1)
out_time_workdays = out_time_workdays.drop(labels=['Unnamed: 0'], axis=1)

Unnamed: 0       0
2015-01-01    4410
2015-01-02     209
2015-01-05     206
2015-01-06     228
              ... 
2015-12-25    4410
2015-12-28     234
2015-12-29     230
2015-12-30     265
2015-12-31     213
Length: 262, dtype: int64
Unnamed: 0       0
2015-01-01    4410
2015-01-02     209
2015-01-05     206
2015-01-06     228
              ... 
2015-12-25    4410
2015-12-28     234
2015-12-29     230
2015-12-30     265
2015-12-31     213
Length: 262, dtype: int64


Now lets convert our in/out timesheet DataFrames to datetime objects for time calculation.

In [9]:
# Set columns to index
workdays_cols  = in_time_workdays.columns

# Convert datatype to datetime on 'in' timesheet DataFrame
in_time_workdays[workdays_cols] = in_time_workdays[workdays_cols]\
.apply(pd.to_datetime, errors='coerce')

# Convert datatype to datetime on 'out' timesheet DataFrame
out_time_workdays[workdays_cols] = out_time_workdays[workdays_cols]\
.apply(pd.to_datetime, errors='coerce')

We can take advantage of the identical configuration of the 'in_time'/'out_time' DataFrames to evaluate the period worked each day by each employee via the .subtract() method. We can then clean the result by replacing missing days (days in which the employee did not clock in) with a timedelta value of 0 seconds.

In [10]:
# Calculate daily worked hours via subtraction
hours_timedelta = out_time_workdays.subtract(in_time_workdays)

# Replace NaT values with 0 for hours worked
hours_timedelta = hours_timedelta.fillna(pd.Timedelta(seconds=0))

Next, we can calculate summary statistics over each employee's worked hours and rename our columns to reflect the calculations. 

In [11]:
# Calculate summary statistics on hours worked
hours_stats = hours_timedelta.T.agg(['mean','max','sum','std'])

# Rename columns
hours_stats = hours_stats.T
hours_stats.columns = ['MeanHrsWorked', 'MaxHrsWorked', 'SumHrsWorked', 'StdHrsWorked']

# Set the index to 'EmployeeID' column for later joining
hours_stats = hours_stats.set_index(gen_data_orig['EmployeeID'])

For easier calculations, we will now convert the timedelta values in our summary statistics DataFrame to floating point numbers in units of hours.

In [12]:
# Convert timedelta to time in hours
for column in hours_stats:
    hours_stats[column] = hours_stats[column].dt.total_seconds()/3600

Back-tracking to the 'in_time' DataFrame, we can quickly calculate the number of sick days/vacation days taken by each employee. This is accomplished by summing the number of days in which the employee did not check in (value is NaT).

In [13]:
# Initialize sick_days DataFrame
sick_days = pd.DataFrame([])

# Sum the number of days for each employee where no check-in time was recorded (NaT)
sick_days['SickDays'] = in_time_workdays.T.isna().sum() 

# Set the index to the common 'EmployeeID' column for joining
sick_days = sick_days.set_index(gen_data_orig['EmployeeID'])

Finally, lets inner-join the four cleaned DataFrames to form our final cleaned DataFrame for export.

In [14]:
# Join the four cleaned DataFrames using an inner-join
df = gen_data.merge(hours_stats, on='EmployeeID')\
    .merge(sick_days, on='EmployeeID')\
        .merge(emp_survey, on='EmployeeID')\
            .merge(man_survey, on='EmployeeID')

Last, lets check out and export our aggregated and cleaned HR dataset.

In [17]:
print(df.info())
print(df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4382 entries, 1 to 4409
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4382 non-null   int64  
 1   Attrition                4382 non-null   object 
 2   BusinessTravel           4382 non-null   object 
 3   Department               4382 non-null   object 
 4   DistanceFromHome         4382 non-null   float64
 5   Education                4382 non-null   int64  
 6   EducationField           4382 non-null   object 
 7   EmployeeCount            4382 non-null   int64  
 8   Gender                   4382 non-null   object 
 9   JobLevel                 4382 non-null   int64  
 10  JobRole                  4382 non-null   object 
 11  MaritalStatus            4382 non-null   object 
 12  MonthlyIncome            4382 non-null   float64
 13  NumCompaniesWorked       4382 non-null   float64
 14  Over18                  

In [16]:
%store df

Stored 'df' (DataFrame)
